In [5]:
import pandas as pd

df = pd.read_csv('../data/full/ratings_edited.csv')
df

,userId,movieId,rating,timestamp,movie_idx
0,0,307,3.5,1256677221,307
1,0,481,3.5,1256677456,481
2,0,1091,1.5,1256677471,1091
3,0,1257,4.5,1256677460,1257
4,0,1449,4.5,1256677264,1449
...,...,...,...,...,...
27753439,283227,8542,4.5,1379882795,8351
27753440,283227,8712,4.5,1379882751,8513
27753441,283227,34405,4.5,1379882889,22224
27753442,283227,44761,4.5,1354159524,27965


In [6]:
import time

time.time(), df.timestamp.max()

(1670649369.5136166, 1537945149)

In [7]:
now = time.time()
def trend_score(movie):
    filtered = df[df.movie_idx == movie]
    avg_time = ((now - filtered.timestamp).sum()/len(filtered))/(1000*60*60*24)
    votes = (filtered.rating - 2.5).sum()
    return (abs(votes) ** 0.2)/(avg_time ** 2) * (1 if votes > 0 else -1)

In [9]:
import requests

def put(movie):
    filtered = df[df.movie_idx == movie]
    score =  trend_score(movie)
    j = {'trend_score': score}
    try:
        requests.put(f'http://localhost:3001/movies/{movie}', json=j)
    except:
        print(movie, score)

movies = set(df.movie_idx.values)
for movie in movies:
    put(movie)